In [1]:
import mmap
import time
from time import strftime, localtime
import glob
import os
import json
from collections import deque
import random

import import_ipynb
import torch
import torch.nn.functional as F
from torch.distributions import Categorical
from torch_geometric.data import Data
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import sys 
import win32pipe, win32file, pywintypes

from actor_no_readout import actor_network

torch.set_printoptions(threshold=10_000)
np.set_printoptions(threshold=sys.maxsize)



BUFFER_SIZE = 200000

c:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 통신 관련 함수

In [2]:
def sendOmnetMessage(msg):
    win32file.WriteFile(pipe, msg.encode('utf-8'))
    
def getOmnetMessage():
    response_byte = win32file.ReadFile(pipe, BUFFER_SIZE)
    response_str = response_byte[1].decode('utf-8')
    return response_str

def closePipe():
    win32file.CloseHandle(pipe)

# 실험 정보 관련 함수

In [3]:
def recordExpInfo(config):

    networkInfo = config["network_info"]

    modelNum = int(networkInfo['modelNum'])
    availableJobNum = int(networkInfo['availableJobNum'])
    nodeNum = int(networkInfo['nodeNum'])
    jobWaitingLength = int(networkInfo['jobWaitingQueueLength'])
    adjacency = eval(networkInfo['adjacencyList'])
    episode_length = int(networkInfo['episode_length'])
    node_capacity = networkInfo['node_capacity']
    job_generate_rate = networkInfo['job_generate_rate']

    node_feature_num = 2 * (modelNum * availableJobNum)
    queue_feature_num = (nodeNum + modelNum) * jobWaitingLength
    hidden_feature_num = 10*(node_feature_num + queue_feature_num)
    reward_weight = 1/modelNum
    entropy_weight = config["entropy_weight"]
    
    info = f"""
    노드 개수 : {nodeNum}
    네트워크 최대 job 개수 : {availableJobNum}
    job 대기 가능 개수 : {jobWaitingLength}
    최대 subtask 개수 : {modelNum}
    인접 리스트 : {adjacency}
    node_feature_num : {node_feature_num}
    queue_feature_num : {queue_feature_num}
    episode_length : {episode_length}
    node_capacity : {node_capacity}
    entropy_weight : {entropy_weight}
    reward_weight : {reward_weight}
    job_generate_rate : {job_generate_rate}
    """
    print(info)

    with open(f'{config["path_name"]}/info.txt', 'w') as f:
        f.write(f'{info}')

In [4]:
random.seed(42)

def main(config):
    global adjacency, writer

    recordExpInfo(config)
    
    model = actor_network(config)
    total_params = sum(p.numel() for p in model.parameters())
    print(total_params)
    model.load_state_dict(torch.load("C:/Users/user/Desktop/suhwan/connection_test/python_agent/experiment/subtask_reward/history2/model_2500.pth"))
    # model.load_state_dict(torch.load("C:/Users/user/Desktop/suhwan/connection_test/python_agent/experiment/subtask_num_test_same_env/history17/model_1100.pth"))
    reward_history = []
    v_history = []

    adjacency = torch.tensor(adjacency, dtype=torch.long)

    step = 1
    episode = 1

    max_reward = 0

    average_reward = 0
    average_reward_num = 0

    temp_history = deque([])
    episode_history = []
    rewards = []
    actions = []

    isStop = False
    node_selected_num = [0 for i in range(config["node_num"])]
    void_selected_num = 0

    pre_state_1 = {}
    pre_state_2 = {}

    void_mask = [0] * config["node_num"] + [1]
    unvoid_mask = [1] * config["node_num"] + [0]

    episode_total_reward = 0
    hidden = (torch.zeros(1, 1, config["lstm_hidden_num"]), torch.zeros(1, 1, config["lstm_hidden_num"]))
    
    while True:
        # time.sleep(config["cpu_load_balance_time"])
        
        model = model.to('cpu')
        model.eval()
        msg = getOmnetMessage()
        
        if msg == "action": # omnet의 메세지, state 받으면 됨
            sendOmnetMessage("ok")
            state_1 = getOmnetMessage()
            state_2 = getOmnetMessage()

            if len(state_1) == 0:
                state_1 = state_2
            

            if len(pre_state_1) == 0: # action 시작
                state_1 = json.loads(state_1) # state 받았으므로 action 하면됨.
                state_2 = json.loads(state_2) # state 받았으므로 action 하면됨.
                
            else:
                state_1 = json.loads(state_1)
                pre_state_1['jobWaiting'] = state_1['jobWaiting']
                pre_state_1['sojournTime'] = state_1['sojournTime']
                state_1 = pre_state_1

                state_2 = json.loads(state_2)
                pre_state_2['jobWaiting'] = state_2['jobWaiting']
                pre_state_2['sojournTime'] = state_2['sojournTime']
                state_2 = pre_state_2
            
            sendOmnetMessage("ok") # 답장

            
            node_waiting_state_1 = torch.tensor(eval(str(state_1['nodeState'])), dtype=torch.float)
            node_processing_state_1 = torch.tensor(eval(state_1['nodeProcessing']), dtype=torch.float)
            link_state_1 = torch.tensor(eval(state_1['linkWaiting']), dtype=torch.float)
            job_waiting_state_1 = torch.tensor(eval(state_1['jobWaiting']), dtype=torch.float)
            activated_job_list_1 = eval(state_1['activatedJobList'])
            isAction_1 = int(state_1['isAction'])
            reward_1 = float(state_1['reward'])
            averageLatency_1 = float(state_1['averageLatency'])
            completeJobNum_1 = int(state_1['completeJobNum'])
            sojournTime_1 = float(state_1['sojournTime'])
            #startLatency_1 = float(state_1["startLatency"])

            node_waiting_state_2 = torch.tensor(eval(str(state_2['nodeState'])), dtype=torch.float)
            node_processing_state_2 = torch.tensor(eval(state_2['nodeProcessing']), dtype=torch.float)
            link_state_2 = torch.tensor(eval(state_2['linkWaiting']), dtype=torch.float)
            job_waiting_state_2 = torch.tensor(eval(state_2['jobWaiting']), dtype=torch.float)
            activated_job_list_2 = eval(state_2['activatedJobList'])
            isAction_2 = int(state_2['isAction'])
            reward_2 = float(state_2['reward'])
            averageLatency_2 = float(state_2['averageLatency'])
            completeJobNum_2 = int(state_2['completeJobNum'])
            sojournTime_2 = float(state_2['sojournTime'])
            #startLatency_2 = float(state_2["startLatency"])

            # print(reward_2)



            # node_waiting_state_2 = torch.tensor(eval(str(state_1['nodeState'])), dtype=torch.float)
            # node_processing_state_2 = torch.tensor(eval(state_1['nodeProcessing']), dtype=torch.float)
            # link_state_2 = torch.tensor(eval(state_1['linkWaiting']), dtype=torch.float)
            # job_waiting_state_2 = torch.tensor(eval(state_1['jobWaiting']), dtype=torch.float)
            # activated_job_list_2 = eval(state_1['activatedJobList'])
            # isAction_2 = int(state_1['isAction'])
            # reward_2 = float(state_1['reward'])
            # averageLatency_2 = float(state_1['averageLatency'])
            # completeJobNum_2 = int(state_1['completeJobNum'])
            # sojournTime_2 = float(state_1['sojournTime'])

            writer.add_scalar("completeJobNum/train", completeJobNum_2 ,step)

            #print(node_waiting_state_2)
            #print(node_processing_state_2)
            

            # # 이 timestep에서 발생한 모든 샘플에 똑같은 보상 적용.
            # if averageLatency_2 == -1:
            #     reward_2 = 0
            # else:
            #     reward_2 = completeJobNum_2

            # reward_2 = startLatency_2 * 100

            #print(reward_2)
            
            writer.add_scalar("Reward/train", reward_2, step)

            episode_total_reward += reward_2

            if reward_2 != 0:
                rewards.append(reward_2)
                
            first_sample = True
            if config["is_train"] and len(pre_state_1) == 0:
                if temp_history:
                    # temp_history[-1][3] = reward_2
                    temp_history[-1][4] = network_state
                    temp_history[-1][5] = job_waiting_state

                while temp_history:
                    history = temp_history.popleft()
                    
                    # model.history.append(history)
                    episode_history.append(history)
                    model.put_data(history)

            

            temp_history = deque([])

            job_index = int(state_2['jobIndex'])

            #print('sojourn time :', sojournTime)


            # node_state_1 = np.concatenate((node_waiting_state_1,node_processing_state_1) ,axis = 1)
            #node_state_1 = torch.concat([node_waiting_state_1, node_processing_state_1], dim=1)
            node_state_1 = node_waiting_state_1
            # node_state_2 = np.concatenate((node_waiting_state_2,node_processing_state_2) ,axis = 1)
            #node_state_2 = torch.concat([node_waiting_state_2, node_processing_state_2], dim=1)
            node_state_2 = node_waiting_state_2

            #print(reward)

            # link_state_1 = torch.tensor(link_state_1, dtype=torch.float)
            # link_state_2 = torch.tensor(link_state_2, dtype=torch.float)

            job_waiting_num = 0
            job_waiting_queue = deque()
            for job in job_waiting_state_2:
                if any(job): # 하나라도 0이 아닌 것 이 있으면 job이 있는것임.
                    job_waiting_num += 1
                    job_waiting_queue.append(job)
            
            job_waiting_state_1 = job_waiting_state_1.view(1, -1)
            job_waiting_state_2 = job_waiting_state_2.view(1, -1)
            # print(job_waiting_state)

            network_state_1 = Data(x=node_state_1, edge_attr=link_state_1, edge_index=adjacency)
            network_state_2 = Data(x=node_state_2, edge_attr=link_state_2, edge_index=adjacency)

            network_state = [network_state_1, network_state_2]
            job_waiting_state = [job_waiting_state_1, job_waiting_state_2]

            pre_state_1 = state_1
            pre_state_2 = state_2
            
            if average_reward_num == 0:
                average_reward = reward_2
                average_reward_num = 1
            else:
                average_reward = average_reward + (reward_2 - average_reward)/(average_reward_num + 1)
                average_reward_num += 1
                
            if step > 1:
                for i in range(config["node_num"]):
                    node_tag = "node/" + str(i) + "/train"
                    writer.add_scalar(node_tag, node_selected_num[i], step)

                writer.add_scalar("node/void/train", void_selected_num, step)
                    
                node_selected_num = [0 for i in range(config["node_num"])] # node selected num 초기화
                void_selected_num = 0

                if reward_2 != 0:
                    with torch.no_grad():
                        state = model.gnn([network_state, job_waiting_state])
                        writer.add_scalar("Value/train", torch.mean(model.v(state)), step)
                

                writer.flush()

            
            
            # print(job_waiting_queue)
            if job_waiting_num == 0:
                isAction_2 = False
                

            if isAction_2:
                """if step % config["T_horizon"] == 0:

                    print("hello")

                    if config["is_train"]:
                        if episode % 100 == 0:
                            tm = localtime(time.time())
                            time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                            print(f"[{time_string}] training....")
                        model.train_net()
                        if episode % 100 == 0:
                            tm = localtime(time.time())
                            time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                            print(f"[{time_string}] training complete")

                        model = model.cpu()"""


                job_idx = job_index
                job = job_waiting_queue.popleft()
                src = -1
                dst = 1
                for i in range(config["node_num"]):
                    if job[i] == -1:
                        src = i
                    if job[i] == 1:
                        dst = i

                if src == -1:
                    src = dst
                    
                #print(f"src : {src}, dst : {dst}")
                #print(job)
                subtasks = job[config["node_num"]:]
                offloading_vector = []
                temp_data = []
                scheduling_start = False
                # print(subtasks)
                step += 1

                with torch.no_grad():
                    feature = model.gnn([network_state, job_waiting_state])
                    feature = F.normalize(feature, dim=1)
                    new_feature = feature.unsqueeze(0)
                    first_prob, entropy, output, hidden = model.pi([new_feature, hidden])

                writer.add_scalar("Entropy/train", torch.mean(entropy).item(), step)
                #print(f'prob : {prob}')

                # isVoid = F.sigmoid(dists[modelNum].sample())

                m = Categorical(first_prob[0][0]) # 첫 번째 batch의 첫 번째 node+void개의 확률들
                nodes = m.sample()

                node = nodes.item()

                #print(f'node : {node}')
                
                # void action 실험용
                # node = nodeNum 
                
                
                # void action 뽑으면 void만 업데이트
                if node == config["node_num"] and not scheduling_start: 
                    actions.append(node)
                    action_mask = void_mask

                    temp_history.append([
                        [network_state[0], network_state[1]], 
                        [job_waiting_state[0], job_waiting_state[1]], 
                        node, 0, 
                        [network_state[0], network_state[1]], 
                        [job_waiting_state[0], job_waiting_state[1]],
                        1, 
                        0, action_mask, 0]
                    )

                    sendOmnetMessage("void")

                    #print("action finish.")
                    
                    if getOmnetMessage() == "ok":
                        void_selected_num += 1

                else:
                    scheduling_start = True

                if scheduling_start:
                    
                    if random.random() > config["imitation_probability"]:
                            config["our"] = True
                    else:
                        config["our"] = False
                    

                    for sub_index in range(config["model_num"]):

                        with torch.no_grad():
                            feature = model.gnn([network_state, job_waiting_state])
                            feature = F.normalize(feature, dim=1)
                            new_feature = feature.unsqueeze(0)
                            prob, entropy, output, hidden = model.pi([new_feature, hidden])
                            # print(prob)

                        writer.add_scalar("Entropy/train", torch.mean(entropy).item(), step)

                        output = output[:, :, 0:-1].squeeze(0)

                        prob = F.softmax(output, dim=1)
                        prob = torch.concat([prob, torch.zeros(1, 1)], dim=1)

                        m = Categorical(prob)
                        nodes = m.sample()
                        action_mask = unvoid_mask


                        if config["our"]:
                            node = nodes[0].item()
                        else:
                            node = torch.argmin(node_waiting_state_2[:,0]).item()

                        next_node_state_2 = node_state_1.clone() # node_state_1을 node_state2로 복사

                        #print(node_state_1)

                        next_node_state_1 = node_state_1.clone()
                        next_node_state_1[node][3] += subtasks[sub_index]
                        next_node_state_1[node][1] = next_node_state_1[node][3] / next_node_state_1[node][2]

                        next_network_state_1 = Data(x=next_node_state_1, edge_attr=link_state_1, edge_index=adjacency)
                        next_network_state_2 = Data(x=next_node_state_2, edge_attr=link_state_2, edge_index=adjacency)

                        next_network_state = [next_network_state_1, next_network_state_2]
                        # allJobWait, allJobWaitTime, power, jobRemain
                        next_job_waiting_state = [job_waiting_state_1, job_waiting_state_2]

                        temp_history.append([
                        [network_state[0], network_state[1]], 
                        [job_waiting_state[0], job_waiting_state[1]], 
                        node, 0, 
                        next_network_state, 
                        next_job_waiting_state,
                        prob[0][node].item(),
                        0, action_mask, 0]
                        )

                        offloading_vector.append(node)
                        node_selected_num[node] += 1
                        actions.append(node)

                        node_state_1 = next_node_state_1.clone()
                        node_state_2 = next_node_state_2.clone()

                        network_state_1 = Data(x=node_state_1, edge_attr=link_state_1, edge_index=adjacency)
                        network_state_2 = Data(x=node_state_2, edge_attr=link_state_2, edge_index=adjacency)

                        network_state = [network_state_1, network_state_2]
                        job_waiting_state = next_job_waiting_state

                if len(offloading_vector) != 0: # for문을 다 돌면 -> void action 안뽑으면
                    # print(offloading_vector)
                    msg = str(offloading_vector)
                    sendOmnetMessage(msg)
                    
                    #print("action finish.")
                    if(getOmnetMessage() == "ok"):
                        pass

        elif msg == "stop":
            
            sendOmnetMessage("ok")
            pre_state_1 = {}
            pre_state_2 = {}
            
        elif msg == "episode_finish":
            sendOmnetMessage("ok")

            # print(len(model.history))

            mean = np.mean(rewards)
            std = np.std(rewards)

            # print(actions)

            #print(mean)
            #print(std)
            #print(rewards)

            #for i in range(len(episode_history)):
            #    if episode_history[i][3] != 0:
            #        episode_history[i][3] = (episode_history[i][3] - mean) / (std + 1e-10)
            #        #print((episode_history[i][3] - mean) / (std + 1e-10))

            rewards = []
            actions = []


            

            

            episodic_reward = getOmnetMessage()
            episodic_reward = json.loads(episodic_reward)
            
            finish_num = float(episodic_reward['reward'])
            complete_num = int(episodic_reward['completNum'])
            average_latency = float(episodic_reward['averageLatency'])
            jitter = episodic_reward['jitter']
            jitterMake = episodic_reward['jitterMake']
            action_id = episodic_reward['action_id']
            action_reward = episodic_reward['action_reward']
            #print(list(map(float, jitter.strip().split(" "))))
            #print(list(map(float, jitterMake.strip().split(" "))))
            #print(list(map(int, action_id.strip().split(" "))))
            #print(list(map(float, action_reward.strip().split(" "))))


            normalized_finish_num = model.return_normalize_reward(finish_num)
            
            writer.add_scalar("EpisodicReward/train", finish_num, episode)
            writer.add_scalar("NormalizedEpisodicReward/train", normalized_finish_num, episode)
            writer.add_scalar("CompleteNum/train", complete_num, episode)
            writer.add_scalar("averageLatency/train", average_latency ,episode)

            # model.history.append(episode_history)
            #aaaa = [episode_history[i][2] for i in range(len(episode_history))]
            #print(len(aaaa))
            #print(aaaa)
            #rrrr = [episode_history[i][3] for i in range(len(episode_history))]
            #print(len(rrrr))
            #print(rrrr)
            
            model.data = episode_history[:]
            data_length = len(model.data)
            reward_list = [(data_length - i) * -0.01 for i in range(data_length)]

            action_id_list = list(map(int, action_id.strip().split(" "))) if action_id != '' else []
            action_reward_list = list(map(float, action_reward.strip().split(" "))) if action_reward != '' else []

            action_id_reward_list = sorted(zip(action_id_list, action_reward_list))

            #print(action_id_reward_list)

            for i in range(len(action_id_reward_list)):
                index = action_id_reward_list[i][0]
                if index < data_length:
                    reward_list[index] = action_id_reward_list[i][1]

            #print(reward_list)

            for i in range(len(model.data)):
                model.data[i][3] = reward_list[i]

            tt = []
            for i in range(len(model.data)):
                tt.append(model.data[i][3])
            #print(tt)
            #print(len(model.data))
            #print(tt)
            episode_history = []
            temp_history = deque([])

            

            episode_total_reward += complete_num

            writer.add_scalar("episode_total_reward/train", episode_total_reward, episode)

            episode_total_reward = 0
            hidden = (torch.zeros(1, 1, config["lstm_hidden_num"]), torch.zeros(1, 1, config["lstm_hidden_num"]))

            episode += 1
            sendOmnetMessage("ok")

            config["entropy_weight"] = max(0.0001, config["entropy_weight"] * config["entropy_gamma"])
            config["imitation_probability"] = max(config["imitation_gamma"] * config["imitation_probability"], 0.2)

            if finish_num > max_reward:
                modelPathName = config["path_name"] + "/max_model.pth"
                torch.save(model.state_dict(), modelPathName)
                max_reward = finish_num

            writer.add_scalar("AverageReward/train", average_reward, step)
            average_reward = 0
            average_reward_num = 0

            if config["is_train"]:
                
                if episode % 100 == 0:
                    tm = localtime(time.time())
                    time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                    print(f"[{time_string}] training....")
                model.train_net()
                if episode % 100 == 0:
                    tm = localtime(time.time())
                    time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                    print(f"[{time_string}] training complete")

                if episode % 100 == 0:
                    tm = localtime(time.time())
                    time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                    print(f"[{time_string}] training replay buffer....")
                model.train_net_history()
                if episode % 100 == 0:
                    tm = localtime(time.time())
                    time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                    print(f"[{time_string}] training complete")

                model.clear_data()

                if episode % 100 == 0:
                    modelPathName = config["path_name"] + "/model.pth"
                    torch.save(model.state_dict(), modelPathName)
                    modelPathName = config["path_name"] + f"/model_{episode}.pth"
                    torch.save(model.state_dict(), modelPathName)

                    time.sleep(10)

                model.eval()
                
                

            

                
                
                
                



# 통신 관련 초기화

In [5]:
PIPE_NAME = "\\\\.\\pipe\\worker_right_latency_sub_trace4"
BUFFER_SIZE = 200000

try:
    pipe = win32pipe.CreateNamedPipe(
        PIPE_NAME,
        win32pipe.PIPE_ACCESS_DUPLEX,
        win32pipe.PIPE_TYPE_MESSAGE | win32pipe.PIPE_READMODE_MESSAGE | win32pipe.PIPE_WAIT,
        1,
        BUFFER_SIZE,
        BUFFER_SIZE,
        0,
        None
    )    
except:
    pass

win32pipe.ConnectNamedPipe(pipe, None)



0

In [6]:
initial_message = getOmnetMessage()
networkInfo = json.loads(initial_message)

modelNum = int(networkInfo['modelNum'])
availableJobNum = int(networkInfo['availableJobNum'])
nodeNum = int(networkInfo['nodeNum'])
jobWaitingLength = int(networkInfo['jobWaitingQueueLength'])
adjacency = eval(networkInfo['adjacencyList'])
episode_length = int(networkInfo['episode_length'])
node_capacity = networkInfo['node_capacity']
job_generate_rate = networkInfo['job_generate_rate']

node_feature_num = 2 * (modelNum * availableJobNum)
queue_feature_num = (nodeNum + modelNum) * jobWaitingLength
hidden_feature_num = 10*(node_feature_num + queue_feature_num)
reward_weight = 1/modelNum

In [7]:
os.chdir("C:/Users/user/Desktop/suhwan/connection_test/python_agent/experiment/subtask_reward")
folderList = glob.glob("history*")

pathName = "history" + str(len(folderList))

print(pathName)

os.mkdir(pathName)

writer = SummaryWriter(pathName)

history8


In [8]:
if __name__ == '__main__':
    sendOmnetMessage("init") # 입력 끝나면 omnet에 전송
    print("네트워크 초기화 완료")

    config = {
        "learning_rate"         : 0.00005,
        "gamma"                 : 0.9,
        "entropy_weight"        : 0.0001,
        "entropy_gamma"         : 0.9998,
        "lambda"                : 0.99,
        "eps_clip"              : 0.08,
        "batch_size"            : 256,
        "loss_coef"             : 0.5,
        "job_generate_rate"     : 0.003,
        "is_train"              : False,
        "replay_buffer_size"    : 100,
        "history_learning_time" : 0,
        "current_learning_time" : 2,
        "node_feature_num"      : 4,
        "queue_feature_num"     : (nodeNum + modelNum) * jobWaitingLength,
        "hidden_feature_num"    : 128,
        "reward_weight"         : 1.0/1,
        "node_num"              : nodeNum,
        "model_num"             : modelNum,
        "lstm_hidden_num"       : 128,
        "cpu_load_balance_time" : 0.1,
        "network_info"          : networkInfo,
        "path_name"             : pathName,
        "T_horizon"             : 1000,
        "link_num"              : 32,
        "state_weight"          : 1.0,
        "our"                   : True,
        "imitation_probability" : 0.0,
        "imitation_gamma"       : 1.0,
    }


    main(config)
            
    



네트워크 초기화 완료

    노드 개수 : 5
    네트워크 최대 job 개수 : 5
    job 대기 가능 개수 : 15
    최대 subtask 개수 : 3
    인접 리스트 : [[0, 1, 0, 2, 1, 2, 1, 3, 2, 3, 2, 4, 3, 4], [1, 0, 2, 0, 2, 1, 3, 1, 3, 2, 4, 2, 4, 3]]
    node_feature_num : 30
    queue_feature_num : 120
    episode_length : 1000
    node_capacity : 0.100000, 0.300000
    entropy_weight : 0.0001
    reward_weight : 0.3333333333333333
    job_generate_rate : 5
    
1429847


c:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\numpy\core\_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
c:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\numpy\core\_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


error: (109, 'ReadFile', '파이프가 끝났습니다.')

In [ ]:
a = [0, 3, 9, 1, 6, 12, 4, 10, 2, 5, 13, 18, 11, 21, 14, 7, 19, 24, 22, 20, 8, 23, 15, 27, 16, 17, 33, 34, 35, 25, 30, 28, 36, 39, 26, 29, 31, 40, 42, 41, 43, 45, 46]
b = [0.027814, 0.036799, 0.074262, 0.033636, 0.029843, 0.040076, 0.033607, 0.040415, 0.087693, 0.030417, 0.025153, 0.131984, 0.043085, 0.098693, 0.03274, 0.013596, 0.033249, 0.241986, 0.036045, 0.085203, 0.0479, 0.078513, 0.022581, 0.093197, 0.028499, 0.106345, 0.023464, 0.041557, 0.11446, 0.008629, 0.034807, 0.009598, 0.027988, 0.053074, 0.079723, 0.03463, 0.010012, 0.026495, 0.098693, 0.027873, 0.028241, 0.062686, 0.030221]


d = sorted(zip(a,b))
print(len(d))
print(sorted(d))

# model.data 개수에 맞춰야함.

43
[(0, 0.027814), (1, 0.033636), (2, 0.087693), (3, 0.036799), (4, 0.033607), (5, 0.030417), (6, 0.029843), (7, 0.013596), (8, 0.0479), (9, 0.074262), (10, 0.040415), (11, 0.043085), (12, 0.040076), (13, 0.025153), (14, 0.03274), (15, 0.022581), (16, 0.028499), (17, 0.106345), (18, 0.131984), (19, 0.033249), (20, 0.085203), (21, 0.098693), (22, 0.036045), (23, 0.078513), (24, 0.241986), (25, 0.008629), (26, 0.079723), (27, 0.093197), (28, 0.009598), (29, 0.03463), (30, 0.034807), (31, 0.010012), (33, 0.023464), (34, 0.041557), (35, 0.11446), (36, 0.027988), (39, 0.053074), (40, 0.026495), (41, 0.027873), (42, 0.098693), (43, 0.028241), (45, 0.062686), (46, 0.030221)]


In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        with record_function("model_inference"):
            for i in range(10):
                print(i)
            
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

In [ ]:
a = torch.randn(2, 5)
print(a)

a = a.unsqueeze(1)
print(a.shape)
print(a)

a = a.repeat(1, 4, 1)
print(a.shape)
print(a)